In [1]:
import pandas as pd
import pdb

from utils.experiment_utils import *

%load_ext autoreload
%autoreload 2

Read in all saved results and make Latex table

Metrics:
* marginal cov
* ACCG
* average set size


To get multiple row levels, set a multi-index. To get multiple column levels, concatenate dataframes along axis=1.

In [3]:
# Map from dataset to folder with saved results
datasets = {'ImageNet': '.cache/paper/imagenet',
            'CIFAR-100': '.cache/paper/cifar100',
            'Places365': '.cache/paper/places365',
            'iNaturalist': '.cache/paper/inaturalist'}

n_totalcals = {10: '$n=10$', 30: '$n=30$'}
score_functions = {'softmax': '$\softmax$',
                  'APS': '$\APS$',
                  'RAPS': '$\RAPS$'}

methods = {'standard': '$\standard$', 
            'classwise': '$\classwise$',
            'always_cluster': '$\clustered$'}


# ATTEMPT 1

vertical_list = []
for dataset, save_folder in datasets.items():
    for score, score_name in score_functions.items():
        for method, method_name in methods.items(): 
            horizontal_dict = {}
            for n_totalcal, n_name in n_totalcals.items():
                folder = f'{save_folder}/n_totalcal={n_totalcal}/score={score}/'
                df = average_results_across_seeds(folder, print_results=False) # df = results DataFrame
                horizontal_dict[n_name] = pd.Series({'Dataset': dataset, 'Score function': score_name, 'Method': method_name, 
                               'ACCG': f"{df[df['method'] == method]['class_cov_gap_mean'].values[0]:.2f}" 
                                       f" ({df[df['method'] == method]['class_cov_gap_se'].values[0]:.2f})",
                               'Avg. set size:': f"{df[df['method'] == method]['avg_set_size_mean'].values[0]:.1f}" 
                                               f" ({df[df['method'] == method]['avg_set_size_se'].values[0]:.1f})",
                               'Marginal coverage:': f"{df[df['method'] == method]['marginal_cov_mean'].values[0]:.2f}" 
                                               f" ({df[df['method'] == method]['marginal_cov_se'].values[0]:.2f})"})
                       
            horizontal_df = pd.concat(horizontal_dict, axis=1).transpose()
#             pdb.set_trace()
            vertical_list.append(horizontal_df)                                         

df = pd.concat(vertical_list, ignore_index=True)                                                     
df = df.set_index(['Dataset', 'Score function', 'Method'])
# print(df)
print(df.to_latex(escape=False # So that backslashes are printed as `\' rather than '\textbackslash'
                 ))

Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se
0,standard,5.226892,0.089969,38.306105,1.251245,1.937735,0.060146,0.900255,0.002882
1,classwise,6.552409,0.138292,41.471177,4.527139,24.847058,3.356368,0.909628,0.002143
2,always_cluster,4.461816,0.259284,26.717701,4.550845,2.949315,0.757161,0.901609,0.010070


Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se
0,standard,5.225853,0.051083,38.215223,0.616720,1.939136,0.036383,0.900391,0.001748
1,classwise,4.156702,0.070181,23.080164,3.566490,5.795095,0.336304,0.905481,0.001729
2,always_cluster,3.407021,0.128531,18.895543,2.346407,2.678786,0.164535,0.904319,0.004158


Number of seeds found: 10


...

,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se
0,standard,5.539789,0.022748,50.000000,1.310354,7.390987,0.115529,0.942645,0.001879
1,classwise,6.651303,0.210487,42.757164,5.460594,41.182646,3.272216,0.908644,0.008017
2,always_cluster,4.642934,0.144482,43.564830,10.621621,9.787497,2.657746,0.925662,0.006588


Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se
0,standard,5.586198,0.030168,50.725137,2.866401,7.477019,0.065477,0.945227,0.001032
1,classwise,4.451199,0.085627,24.976343,2.059671,15.176454,0.518972,0.905187,0.005257
2,always_cluster,3.825681,0.129089,25.747779,4.840127,14.290644,2.425705,0.921317,0.005926


\begin{tabular}{llllll}
\toprule
            &         &              &         ACCG & Avg. set size: & Marginal coverage: \\
Dataset & Score function & Method &              &                &                    \\
\midrule
ImageNet & $\softmax$ & $\standard$ &  5.23 (0.09) &      1.9 (0.1) &        0.90 (0.00) \\
            &         & $\standard$ &  5.23 (0.05) &      1.9 (0.0) &        0.90 (0.00) \\
            &         & $\classwise$ &  6.55 (0.14) &     24.8 (3.4) &        0.91 (0.00) \\
            &         & $\classwise$ &  4.16 (0.07) &      5.8 (0.3) &        0.91 (0.00) \\
            &         & $\clustered$ &  4.46 (0.26) &      2.9 (0.8) &        0.90 (0.01) \\
            &         & $\clustered$ &  3.41 (0.13) &      2.7 (0.2) &        0.90 (0.00) \\
            & $\APS$ & $\standard$ &  2.57 (0.02) &     25.6 (0.8) &        0.90 (0.00) \\
            &         & $\standard$ &  2.61 (0.01) &     26.1 (0.3) &        0.90 (0.00) \\
            &         & $\classwise$

/tmp/ipykernel_968166/2196600522.py:42: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(escape=False # So that backslashes are printed as `\' rather than '\textbackslash'


In [27]:
# ATTEMPT 2

horizontal_dict = {}
for n_totalcal, n_name in n_totalcals.items():
    vertical_list = []
    for dataset, save_folder in datasets.items():
        for score, score_name in score_functions.items():
            for method, method_name in methods.items(): 

                folder = f'{save_folder}/n_totalcal={n_totalcal}/score={score}/'
                df = average_results_across_seeds(folder, print_results=False, display_table=False) # df = results DataFrame
                s = pd.Series({'Dataset': dataset, 'Score func.': score_name, 'Method': method_name, 
                               'ACCG': f"{df[df['method'] == method]['class_cov_gap_mean'].values[0]:.1f}" 
                                       f" ({df[df['method'] == method]['class_cov_gap_se'].values[0]:.1f})",
                               'Avg. set size': f"{df[df['method'] == method]['avg_set_size_mean'].values[0]:.1f}" 
                                               f" ({df[df['method'] == method]['avg_set_size_se'].values[0]:.1f})",
                               'Marginal cov.': f"{df[df['method'] == method]['marginal_cov_mean'].values[0]*100:.1f}" 
                                                   f" ({df[df['method'] == method]['marginal_cov_se'].values[0]*100:.1f})"})
                vertical_list.append(pd.DataFrame(s).transpose())                                        

    df = pd.concat(vertical_list, ignore_index=True)                                                     
    df = df.set_index(['Dataset', 'Score func.', 'Method'])
    horizontal_dict[n_name] = df
#     pdb.set_trace()

df = pd.concat(horizontal_dict, axis=1)
df_str = df.to_latex(multicolumn_format='c', # To center multilevel column headers
    escape=False # So that backslashes are printed as `\' rather than '\textbackslash'
                 )
# Post-process
df_str = df_str.replace('lllllllll', 'lllcrccrc')
df_str = df_str.replace('\multicolumn{3}{c}{$n=10$} & \multicolumn{3}{c}{$n=30$} \\',
               '\multicolumn{3}{c}{$n=10$} & \multicolumn{3}{c}{$n=30$} \\\\ \cmidrule(lr){4-6}\cmidrule(lr){7-9} \\')
df_str = df_str.replace('CIFAR-100', '\midrule\nCIFAR-100')
df_str = df_str.replace('Places365', '\midrule\nPlaces365')
df_str = df_str.replace('iNaturalist', '\midrule\niNaturalist')
print(df_str)

\begin{tabular}{lllrrrrrr}
\toprule
            &         &              & \multicolumn{3}{c}{$n=10$} & \multicolumn{3}{c}{$n=30$} \\ \cmidrule(lr){4-6}\cmidrule(lr){7-9} \\
            &         &              &       ACCG & Avg. set size & Marginal cov. &       ACCG & Avg. set size & Marginal cov. \\
Dataset & Score func. & Method &            &               &               &            &               &               \\
\midrule
ImageNet & $\softmax$ & $\standard$ &  5.2 (0.1) &     1.9 (0.1) &    90.0 (0.3) &  5.2 (0.1) &     1.9 (0.0) &    90.0 (0.2) \\
            &         & $\classwise$ &  6.6 (0.1) &    24.8 (3.4) &    91.0 (0.2) &  4.2 (0.1) &     5.8 (0.3) &    90.5 (0.2) \\
            &         & $\clustered$ &  4.5 (0.3) &     2.9 (0.8) &    90.2 (1.0) &  3.4 (0.1) &     2.7 (0.2) &    90.4 (0.4) \\
            & $\APS$ & $\standard$ &  2.6 (0.0) &    25.6 (0.8) &    89.9 (0.2) &  2.6 (0.0) &    26.1 (0.3) &    90.1 (0.1) \\
            &         & $\classwise$ &  6.6 (0

/tmp/ipykernel_968166/205200793.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_str = df.to_latex(multicolumn_format='c', # To center multilevel column headers
